# LANL Earthquake Prediction Kaggle Competition 2019
### Eric Yap, Joel Huang, Kyra Wang

---

In this notebook, we present our work for the LANL Earthquake Prediction Kaggle Competition 2019. The goal of this competition is to use seismic signals to predict the timing of laboratory earthquakes. The data comes from a well-known experimental set-up used to study earthquake physics. The `acoustic_data` input signal is used to predict the time remaining before the next laboratory earthquake (`time_to_failure`).

The training data is a single, continuous segment of experimental data. The test data consists of a folder containing many small segments. The data within each test file is continuous, but the test files do not represent a continuous segment of the experiment; thus, the predictions cannot be assumed to follow the same regular pattern seen in the training file.

For each `seg_id` in the test folder, we need to predict a single `time_to_failure` corresponding to the time between the last row of the segment and the next laboratory earthquake.

---

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

# Data wrangling imports
import numpy as np
import pandas as pd

# Utility imports
import ast
from tqdm import tqdm, tqdm_notebook
from joblib import Parallel, delayed

# Data visualization imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import seaborn as sns

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms

# Custom stuff
from series_data import LANLDataset, FeatureGenerator

# Setting the seeds for reproducibility
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
else:
    torch.manual_seed_all(42)

### Data preprocessing

As the training data and the test data are formatted differently, we must either preprocess the data such that the formats of both sets are the same, or ensure that our model is capable of predicting on the two different formats. We went with the first option because it is less time consuming to implement.

We did this by splitting the training data into segments the same size as the test data segments, i.e. 150000 data points each. Each segment is labeled with a single `time_to_failure` corresponding to the time between the last row of the segment and the next laboratory earthquake. We then put each of these segments into a single dataframe, and saved this as a pickle file to be used as our training data.

Following this, we merged the separate test segments into another single dataframe, and saved this as a pickle file to be used as our test data.

As the dataset is massive, we used Joblib to help run the functions as a pipeline jobs with parallel computing.

In [2]:
trainval_df = pd.read_pickle('./data/train_features.pkl')
trainval_df = trainval_df[:-2]
train_segments = np.array([seg for seg in trainval_df['segment'].to_numpy()])
train_labels = trainval_df['target']

In [3]:
test_df = pd.read_pickle('./data/test_features.pkl')

At this point, we split the training data further into a 80/20 training/validation split. We then create dataloaders that will help load the data into the model in parallel using multiprocessing workers.

# Time Series Feature Extraction
https://www.kaggle.com/gpreda/lanl-earthquake-eda-and-prediction

In [68]:
def create_features(seg_id, seg, X):
    xc = pd.Series(seg)
    zc = np.fft.fft(xc)
    
    #FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X.loc[seg_id, 'Rmean'] = realFFT.mean()
    X.loc[seg_id, 'Rstd'] = realFFT.std()
    X.loc[seg_id, 'Rmax'] = realFFT.max()
    X.loc[seg_id, 'Rmin'] = realFFT.min()
    X.loc[seg_id, 'Imean'] = imagFFT.mean()
    X.loc[seg_id, 'Istd'] = imagFFT.std()
    X.loc[seg_id, 'Imax'] = imagFFT.max()
    X.loc[seg_id, 'Imin'] = imagFFT.min()
    X.loc[seg_id, 'Rmean_last_5000'] = realFFT[-5000:].mean()
    X.loc[seg_id, 'Rstd__last_5000'] = realFFT[-5000:].std()
    X.loc[seg_id, 'Rmax_last_5000'] = realFFT[-5000:].max()
    X.loc[seg_id, 'Rmin_last_5000'] = realFFT[-5000:].min()
    X.loc[seg_id, 'Rmean_last_15000'] = realFFT[-15000:].mean()
    X.loc[seg_id, 'Rstd_last_15000'] = realFFT[-15000:].std()
    X.loc[seg_id, 'Rmax_last_15000'] = realFFT[-15000:].max()
    X.loc[seg_id, 'Rmin_last_15000'] = realFFT[-15000:].min()
    
    X.loc[seg_id, 'mean_change_abs'] = np.mean(np.diff(xc))
    X.loc[seg_id, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0])
    
    for windows in [10, 100]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values
        
        X.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

rows = 150000
segments = train_segments.shape[0]

train_features = pd.DataFrame(index=range(segments), dtype=np.float64)
        
for seg_id in tqdm_notebook(range(segments)):
    seg = train_segments[seg_id]
    create_features(seg_id, seg, train_features)

/home/capstone/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/home/capstone/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in true_divide


In [80]:
normalized_features = ((train_features - train_features.mean()) / train_features.std()).values

In [3]:
normalized_features = np.load('./data/normalized_time_series_features.npy')
print(normalized_features.shape)

(4193, 62)


# Data Loading

In [4]:
# Parameters
batch_size = 8
params = {'batch_size': batch_size,
          'shuffle': False,
          'num_workers': 16}

trainval_set = LANLDataset(normalized_features, train_labels, spec=False)
train_set, val_set = trainval_set.train_val_split(0.7, 0.3)

datasets = {
    'train': train_set,
    'val' : val_set
}

dataloaders = {phase: data.DataLoader(dataset, **params)
               for phase, dataset in datasets.items()}

### Defining the Model

In [5]:
class LANLModel(nn.Module):
    def __init__(self, device, input_dim=1, hidden_dim=64, output_dim=1, batch_size=64, num_layers=1):
        super(LANLModel, self).__init__()
        self.device = device
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.rnn = nn.LSTMCell(self.input_dim, self.hidden_dim)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)
        self.to(self.device)
        
    def init_hidden(self, batch_size):
        return (
            torch.zeros(batch_size, self.hidden_dim).to(self.device),
            torch.zeros(batch_size, self.hidden_dim).to(self.device)
        )
        
    def forward(self, x, hc):
        hc = self.rnn(x, hc)
        out = self.fc(hc[0])
        return out, hc

### Training the Model

In [6]:
device = torch.device('cuda')
feature_size = normalized_features.shape[1]

model_params = {
    "batch_size": batch_size,
    "input_dim": feature_size,
    "hidden_dim": 48,
    "num_layers": 1
}

model = LANLModel(device, **model_params)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

num_epochs = 30

for epoch in range(num_epochs + 1):
    model.train()
    train_loss = 0
    hc = model.init_hidden(batch_size)
    for idx, batch in enumerate(dataloaders['train']):
        if idx == (len(dataloaders['train']) - 1):
            continue
        model.zero_grad()
        features = batch['data'].to(device)
        target = batch['target'].to(device)
        output, hc = model(features, hc)
        loss = criterion(output.float(), target)
        loss.backward(retain_graph=True)
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(dataloaders['train'])
    print("Epoch {} | Training loss: {}".format(epoch, train_loss))
    
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        hc = model.init_hidden(batch_size)
        for idx, batch in enumerate(dataloaders['val']):
            if idx == (len(dataloaders['val']) - 1):
                continue
            features = batch['data'].to(device)
            target = batch['target'].to(device)
            output, hc = model(features, hc)
            loss = criterion(output.float(), target)
            validation_loss += loss.item()
    validation_loss /= len(dataloaders['val'])
    print("Epoch {} | Validation loss: {}".format(epoch, validation_loss))

Epoch 0 | Training loss: 3.116730740349689
Epoch 0 | Validation loss: 3.0181773584100267
Epoch 1 | Training loss: 3.0453256907839865
Epoch 1 | Validation loss: 3.0175540696216534
Epoch 2 | Training loss: 3.0386348660700326
Epoch 2 | Validation loss: 3.018032180357583
Epoch 3 | Training loss: 3.0359601954998046
Epoch 3 | Validation loss: 3.0195395003391217
Epoch 4 | Training loss: 3.0315658178901153
Epoch 4 | Validation loss: 3.0178465843200684
Epoch 5 | Training loss: 3.0307984313133303
Epoch 5 | Validation loss: 3.0202498632141306
Epoch 6 | Training loss: 3.026304320353578
Epoch 6 | Validation loss: 3.0197211677515052
Epoch 7 | Training loss: 3.026444337673343
Epoch 7 | Validation loss: 3.0217264528515972
Epoch 8 | Training loss: 3.0237721864144222
Epoch 8 | Validation loss: 3.0200215065026583
Epoch 9 | Training loss: 3.0203142578660307
Epoch 9 | Validation loss: 3.0237030251116694
Epoch 10 | Training loss: 3.018739834143615
Epoch 10 | Validation loss: 3.022883661185639
Epoch 11 | Tra

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/capstone/anaconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-d546d00c82c8>", line 29, in <module>
    loss.backward(retain_graph=True)
  File "/home/capstone/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/tensor.py", line 102, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/capstone/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/autograd/__init__.py", line 90, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/capstone/anaconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'Ke

KeyboardInterrupt: 

### Evaluating the Model on the Test Data